In [ ]:
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt

red_wine = pd.read_excel('winequality-red.xlsx', skiprows = 1)
white_wine = pd.read_excel('winequality-white.xlsx', skiprows = 1)

red_wine['type'] = 'red'
white_wine['type'] = 'white'

both_wine = pd.concat([red_wine, white_wine], ignore_index=True)

In [ ]:
# To transform the categorical data into numeric values we can look at the 'type' column in our data to initialize them as 1 and 2, using map.
# 1 being Red Wine
# 2 being White Wine
both_wine['type_code'] = both_wine['type'].map({'red': 1, 'white': 2})

In [ ]:
numeric_cols = both_wine.select_dtypes(include='number').columns

In [ ]:
both_wine

In [ ]:
both_wine.groupby('type')['quality'].mean()

In [ ]:
both_wine.groupby('type')['quality'].max()

In [ ]:
both_wine.drop(columns='type_code').corr(numeric_only=True)

This is a correlation matrix for all the numeric features in your both_wine DataFrame. Each cell shows the Pearson correlation coefficient between two variables, ranging from –1 (perfect inverse linear relationship) to +1 (perfect direct linear relationship).

Examining this data for key takeaways we can safely assume that alcohol and quality columns have the strongest correlation in the dataset. Meaning the higher the alcohol content of the wine the higher its quality is usually, - which is something everyone can already agree with. 

In [ ]:
both_wine[['alcohol', 'quality']].corr()

This sentiment can also be represented using a scatterplot: 

In [ ]:
plt.figure(figsize=(6, 5))
sns.scatterplot(x='alcohol', y='quality', hue='type', data=both_wine, alpha=0.5)
plt.title('Alcohol vs. Quality')
plt.show()

The overall trend is clear: higher alcohol content generally corresponds with higher quality, visible from how more dots (clustering) appear in the upper-right portion of the chart. The few high-quality wines (8–9) tend to have higher alcohol content. 
This pattern shows a general upward trend: as you go right (more alcohol), the dots shift higher up on the graph (higher quality).

If you look around 12–14% alcohol, there are lots of dots around quality 7 or 8. If you look at 8–10% alcohol, most dots are around quality 3 to 5.

This tells us that wines with more alcohol are more likely to get better scores.

In [ ]:
desc_stats = both_wine.describe()
print(desc_stats)

In [ ]:
for col in numeric_cols:
    plt.figure()
    stats.probplot(both_wine[col].dropna(), dist="norm", plot=plt)
    plt.title(f"Q–Q Plot of {col}")
    plt.show()


Examining the Q-Q plot diagrams for the data as numeric values we can generally assume that if the data-points are roughly on the 45 degree line, we can assume that the data is normal - as seen in for example in pH, density and quality, meaning that the data is "safe" and can be worked with under normality assumptions. Volatile acidity for example has more of an S-shaped curve indicating a deviation from normality.

What the fuck does this actually mean? Essentially if the data is normal it means there are no extreme outliers, no Michael Jordans (salary class example) in the data. So its assumed safe to work with. The opposite is true if the data isn't normal, and we may need a more in depth understanding of the data i.e. see who the fuck the Michael Jordan/Jordans in our data is/are.

Knowing that quality has no crazy outliers we can represent quality by winetype and check which wine usually has better quality using a boxplot: 

In [ ]:
plt.figure(figsize=(6, 5))
sns.boxplot(x='type', y='quality', data=both_wine)
plt.title('Wine Quality by Type')
plt.show()

Generally we can see that there is no considerable difference between red and white wine in terms of quality, although if you zoom in using a microscope you can see that white wine is still a small bit higher, and we can also confirm this by calculating the mean.

In [ ]:
both_wine.groupby('type')['quality'].mean()

We saw in the correlation matrix that the higher the alcohol the higher the quality. We can now group the wines by alcohol, and see which type wine usually has higher alcohol percentage. We expect the result of this to be that white wine usually has the higher alcohol percentage which will further support the rest of our data. 

In [ ]:
plt.figure(figsize=(6, 5))
sns.boxplot(x='type', y='alcohol', data=both_wine)
plt.title('Alcohol Content by Wine Type')
plt.show()

The result is as hypothesized. White wine tends to be rated higher in terms of quality and tends to have a higher alcohol percentage.

In [ ]:
both_wine.groupby('type')['alcohol'].mean()

In [ ]:
plt.figure(figsize=(6, 5))
sns.boxplot(x='type', y='residual sugar', data=both_wine)
plt.title('Residual Sugar by Wine Type')
plt.show()

White wine has significantly more residual sugar but as seen in our correlation matrix residual sugar does not impact the quality of wine significantly. It's pearson correlation being 0.036980, so very little to nothing impact. We can check this further using a scatterplot:

In [ ]:
plt.figure(figsize=(6, 5))
sns.scatterplot(x='residual sugar', y='quality', hue='type', data=both_wine, alpha=0.5)
plt.title('Residual Sugar vs. Quality')
plt.show()

Binning data essentially means slicing the data into short bits. Slicing the cake into pieces and looking at each piece individually. When we slice pH into 5 and 10 bins, we in the following code examine which slice has the heavist wines (highest average density) 

In [ ]:
# 5 bins
both_wine['pH_bin_5'] = pd.cut(both_wine['pH'], bins=5)
density_by_ph_5 = both_wine.groupby('pH_bin_5', observed=True)['density'].mean()
print("Density per pH bin (5 bins):")
print(density_by_ph_5)
print(f"Highest density (5 bins): {density_by_ph_5.idxmax()} with value {density_by_ph_5.max()}")

# 10 bins
both_wine['pH_bin_10'] = pd.cut(both_wine['pH'], bins=10)
density_by_ph_10 = both_wine.groupby('pH_bin_10', observed=True)['density'].mean()
print("\nDensity per pH bin (10 bins):")
print(density_by_ph_10)
print(f"Highest density (10 bins): {density_by_ph_10.idxmax()} with value {density_by_ph_10.max()}")


Using 5 bins we notice that the highest avg. density is in the "3.494 - 3.752" with a value of 0.994.

Using 10 bins means we have more data to work with (slicing the cake into more pieces) and we see that the pH range "3.365 - 3.494" has the highest avg. density. 

5 bins gave a broad peak (3.494–3.752), but 10 bins pinpointed the true maximum/optimal pH range for density (3.365–3.494).

What does this actually mean? 

It means that wine density peaks at a mid-range pH (3.365–3.494), not at the highest pH. 

In [ ]:
both_wine[['pH', 'density']].corr()

In [ ]:
both_wine[['density', 'quality']].corr()

In [ ]:
both_wine[['density', 'alcohol']].corr()
